In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import Pool

In [2]:
from zac_pyutils import ExqUtils
import pandas as pd

In [ ]:
fileIter = ExqUtils.load_file_as_iter("./data/nlp/sample_data.txt")
def func(iter_list): return [i.split("\t") for i in iter_list]
ExqUtils.map_on_iter(fileIter,func,chunk_size=5)

In [9]:
?pd.read_csv

In [29]:
df_chunkList = pd.read_csv("./data/nlp/sample_data.txt", 
            delimiter="\t",
            names=['id','label','weight','feature'],
            chunksize=5,
#             iterator=False,
           )

def my_test(a,b): return str(a)+"\t"+str(b)

for chunk in df_chunkList:
    print("chunk: \n")
    chunk['feature'] = chunk['feature'].map(lambda x: x.split(","))
    chunk['id+label'] = chunk.apply(lambda row: my_test(row['id'],row['label']),axis=1)
    chunk

chunk: 



,id,label,weight,feature,id+label
0,1547316214767546,0,1.0,"[-2.1138265782272E11, 1.9136756786176E11, 1.37...",1547316214767546\t0
1,1547359265142185,1,1.5,"[-3.332587574553254E10, 1.2432920995661206E11,...",1547359265142185\t1
2,1547359265142185,0,1.0,"[-3.332587574553254E10, 1.2432920995661206E11,...",1547359265142185\t0
3,1547395712913576,1,1.0,"[-3.536337856672146E10, 1.7456384445954337E11,...",1547395712913576\t1
4,1547395712913576,0,1.0,"[-3.536337856672146E10, 1.7456384445954337E11,...",1547395712913576\t0


chunk: 



,id,label,weight,feature,id+label
5,1547479463982752,1,1.5,"[-5.4163746911075E10, 9.795769548778902E10, 8....",1547479463982752\t1
6,1547492493344249,1,1.0,"[-5.078099235485994E10, 1.4369949748783133E11,...",1547492493344249\t1
7,1547599855726501,1,1.5,"[-3.190228308965517E10, 1.869087108825747E11, ...",1547599855726501\t1
8,1547649573536780,1,1.5,"[-5.081451975792308E10, 6.6373391832E10, 9.546...",1547649573536780\t1
9,1547649573536780,1,1.0,"[-5.081451975792308E10, 6.6373391832E10, 9.546...",1547649573536780\t1
